## Análisis exploratorio general de la base de datos de RH-IBM

In [189]:
import pandas as pd
from sqlalchemy import create_engine

df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

#motor
engine = create_engine("sqlite:///empleados.db", echo=False)

#df como tabla sql 'empleados'
df.to_sql("empleados", con=engine, index=False, if_exists="replace")

%load_ext sql

%sql sqlite:///empleados.db

In [169]:
#Número total de empleados de la compañía
%sql SELECT COUNT(*) AS total_empleados FROM empleados;

 * sqlite:///empleados.db
Done.


total_empleados
1470


La base de datos tiene registro de 1470 empleados.

* Porcentaje de empleados por sexo:

In [170]:
%sql SELECT Gender , COUNT(*) * 100.0 / (SELECT COUNT(*) FROM empleados) as pct_employees FROM empleados GROUP BY Gender

 * sqlite:///empleados.db
Done.


Gender,pct_employees
Female,40.0
Male,60.0


El 60% de los empleados de la compañía son hombres y el 40% son mujeres.

* ¿Cuál es el estatus marital de los empleados de la compañía?

In [171]:
%%sql 
SELECT 
    MaritalStatus,
    ROUND(COUNT(*) *100.0 / (SELECT COUNT(*) FROM empleados),2) as pct_employees
FROM empleados
GROUP BY MaritalStatus
ORDER BY pct_employees DESC

 * sqlite:///empleados.db
Done.


MaritalStatus,pct_employees
Married,45.78
Single,31.97
Divorced,22.24


El 45.78% de los empleados de la compañía son casados, el 31.97% solteros y el 22.24% son divorciados.

* ¿Qué nivel educativo tienen los empleados de la compañía? 

In [172]:
%%sql 
SELECT
    Education,
    CASE WHEN Education = 1 THEN 'Below College'
         WHEN Education = 2 THEN 'College'
         WHEN Education = 3 THEN 'Bachelor'
         WHEN Education = 4 THEN 'Master' ELSE 'Doctor' END AS EduLevel,
    COUNT(*)as total_employees,
    ROUND(COUNT(*) *100.0 / (SELECT COUNT(*) FROM empleados),2) as pct_employees
FROM empleados
GROUP BY Education
ORDER BY pct_employees DESC

 * sqlite:///empleados.db
Done.


Education,EduLevel,total_employees,pct_employees
3,Bachelor,572,38.91
4,Master,398,27.07
2,College,282,19.18
1,Below College,170,11.56
5,Doctor,48,3.27


El 38.91% de los empleados de la compañía cursaron una licenciatura, el 27.07% tienen una maestria, el 19.18% tiene estudios universitarios y el 3.27 tiene estudios de doctorado. Tan solo el 11.56% de los empleados de la compañía no tiene estudios superiores.

* Campos de estudios de los empleados de la compañía:



In [173]:
%%sql 
SELECT
    EducationField,
    COUNT(*)as total_employees,
    ROUND(COUNT(*) *100.0 / (SELECT COUNT(*) FROM empleados),2) as pct_employees
FROM empleados
GROUP BY EducationField
ORDER BY pct_employees DESC

 * sqlite:///empleados.db
Done.


EducationField,total_employees,pct_employees
Life Sciences,606,41.22
Medical,464,31.56
Marketing,159,10.82
Technical Degree,132,8.98
Other,82,5.58
Human Resources,27,1.84


El 41.22% de los empleados de la compañía tienen estudios relacionados con ciencias de la vida, el 31.56% estudios en medicina, el 10.82% estudios en marketing, el 8.98% estudios en campos técnicos y el 1.84% en recursos humanos.

* Número de empleados que tienen viajes de negocios con frecuencia:

In [174]:
%%sql
SELECT
    BusinessTravel,
    COUNT(*) AS total_employees
FROM empleados
GROUP BY BusinessTravel
ORDER BY total_employees DESC

 * sqlite:///empleados.db
Done.


BusinessTravel,total_employees
Travel_Rarely,1043
Travel_Frequently,277
Non-Travel,150


277 de los empleados de la compañía tienen viajes de negocios con frecuencia.

* Departamentos a los que están vinculados los empleados de la compañía:

In [175]:
%%sql
SELECT 
    Department,
    ROUND(Count(*) *100.0/(SELECT COUNT(*) FROM empleados),2) as pct_employees
FROM empleados
GROUP BY Department
ORDER BY pct_employees DESC

 * sqlite:///empleados.db
Done.


Department,pct_employees
Research & Development,65.37
Sales,30.34
Human Resources,4.29


El 65% de los de empleados en la base da datos hacen parte del departamento de investigación y desarrollo , el 30% a ventas y el 4% a recursos humanos.

* Roles labolares que desempeñan los empleados de la compañía:

In [176]:
%%sql
SELECT
    JobRole,
    ROUND(COUNT(*) * 100.0/(SELECT COUNT(*) FROM empleados),2) as pct_employees
FROM empleados
GROUP BY JobRole
ORDER BY pct_employees DESC

 * sqlite:///empleados.db
Done.


JobRole,pct_employees
Sales Executive,22.18
Research Scientist,19.86
Laboratory Technician,17.62
Manufacturing Director,9.86
Healthcare Representative,8.91
Manager,6.94
Sales Representative,5.65
Research Director,5.44
Human Resources,3.54


Alrededor del 60% de los empleados de la compañía se desempeñan como ejecutivos de ventas, cientificos investigadores o técnicos de laboratorio.

* Teniendo en cuenta todos los roles que desepeñan los empleados de la compañía, ¿Cuál rol tiene el quinto mayor salario mensual promedio?

In [177]:
%%sql
SELECT
    JobRole,
    ROUND(AVG(MonthlyIncome),2) AS Avg_Salary
FROM empleados
GROUP BY JobRole
ORDER BY Avg_Salary DESC
LIMIT 1 OFFSET 4

 * sqlite:///empleados.db
Done.


JobRole,Avg_Salary
Sales Executive,6924.28


El rol con el quinto mayor salario mensual promedio es el de Ejecutivo de ventas con un salario de $ 6924.28.

* ¿Cuál es el Top3 de roles con mayor salario mensual promedio?

In [178]:
%%sql
SELECT JobRole
FROM (
    SELECT JobRole , RANK() OVER(ORDER BY AVG(MonthlyIncome) DESC) AS rank
    FROM empleados
    GROUP BY JobRole) x
WHERE rank <=3

 * sqlite:///empleados.db
Done.


JobRole
Manager
Research Director
Healthcare Representative


Los tres roles con mayor salario mensual promedio son: Gerente, Director de Investigación y Representante de atención medica.

* Cuál es la satisfacción general promedio de los empleados de la compañía? (job, environment, relationship)

In [179]:
%%sql
SELECT Department,
      ROUND((avg(EnvironmentSatisfaction) + avg(JobSatisfaction) + avg(RelationshipSatisfaction)) / 3.0,2) AS Avg_Satisfaction
FROM empleados
GROUP BY Department

 * sqlite:///empleados.db
Done.


Department,Avg_Satisfaction
Human Resources,2.72
Research & Development,2.73
Sales,2.71


Considerando las tres variables, la satisfacción general promedio es cercana a 2.7 para todos los departamentos.

* ¿Qué porcentaje de los empleados de la compañía realiza horas extras?

In [180]:
%%sql
SELECT
    Overtime,
    ROUND(count(*)*100.0/(SELECT COUNT(*) FROM empleados),2) AS pct_employees
FROM empleados
GROUP BY OverTime
    

 * sqlite:///empleados.db
Done.


OverTime,pct_employees
No,71.7
Yes,28.3


El 28.3% de los empleados de la compañía realiza horas extra.

* ¿Qué porcentaje de empleados no tiene la opción de adquirir acciones de la compañía (0)?

In [181]:
%%sql
SELECT ROUND(count(*)*100.0/(SELECT COUNT(*) FROM empleados),2) AS pct_employees
FROM empleados
WHERE StockOptionLevel = 0
GROUP BY StockOptionLevel

 * sqlite:///empleados.db
Done.


pct_employees
42.93


El 42.93% de los empleados de la compañía no tiene opción de adquirir sus acciones.
* ¿Cómo es la rotación de empleados en la compañía? ¿Cuántos empleados se han marchado? ¿Cuántos se han quedado?

In [182]:
%sql SELECT Attrition, count(*) FROM empleados GROUP BY Attrition;

 * sqlite:///empleados.db
Done.


Attrition,count(*)
No,1233
Yes,237


Del total de registros en la base de datos, 1233 empleados permanecen y 237 empleados han abandonado la compañía.
* ¿Cuál es la tasa de rotación de empleados en la compañía?

In [183]:
%%sql
SELECT
    ROUND(
        (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(*),
        2
    ) AS attrition_rate
FROM empleados;

 * sqlite:///empleados.db
Done.


attrition_rate
16.12


La tasa de rotación de empleados es del 16,12%. 
* ¿Cuál es la tasa de rotación por departamento?

In [184]:
%%sql
SELECT 
    Department,  
    ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
    2
   ) AS Attrition_rate
FROM empleados
GROUP BY Department
ORDER BY Attrition_rate DESC

 * sqlite:///empleados.db
Done.


Department,Attrition_rate
Sales,20.63
Human Resources,19.05
Research & Development,13.84


La tasa de rotación más elevada se presenta en el departamento de ventas con un 20.63%, seguido por un 19.05% en el departamento de recursos humanos y 13.84% en investigación y desarrollo.

* ¿Cuál es la tasa de rotación de aquellos que realizan horas extra?

In [185]:
%%sql
SELECT 
    ROUND(SUM(CASE WHEN Attrition ='Yes' THEN 1 ELSE 0 END) *100.0 / COUNT(*),2) AS pct_employees
FROM empleados
WHERE OverTime ='Yes'


 * sqlite:///empleados.db
Done.


pct_employees
30.53


El 30.53% de los empleados que registran haber realizado horas extras han abandonado la compañía.

* ¿Cuál es la tasa de rotación de la mujeres en el departamento de investigación y desarrollo?

In [186]:
%%sql
SELECT 
    ROUND(SUM(CASE WHEN Attrition ='Yes' THEN 1 ELSE 0 END) *100.0 / COUNT(*),2) AS pct_employees
FROM empleados
WHERE Gender='Female' and Department ='Research & Development'


 * sqlite:///empleados.db
Done.


pct_employees
11.35


La tasa de rotación de las mujeres en el departamento de investigación y desarrollo es del 11.35%.

* ¿Cuál es el número promedio de años en que un empleado ejerce como director de manufactura en el departamento de investigación y desarrollo ?

In [187]:
%%sql
SELECT JobRole, ROUND(avg(YearsInCurrentRole),2) AS avg_yearscurrentrole
FROM empleados
where Department = 'Research & Development' and JobRole = 'Manufacturing Director'
GROUP BY JobRole
ORDER BY avg_yearscurrentrole DESC

 * sqlite:///empleados.db
Done.


JobRole,avg_yearscurrentrole
Manufacturing Director,4.97


En promedio un empleado ejerce como director de manufactura en el departamento de investigación y desarrollo 4.97 años.